In [5]:
!pip -q install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# !pip -q install pandas ftfy emoji tqdm deep-translator
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# ====================== DEPENDENCIAS ======================
import re, html, unicodedata, time, threading
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

import pandas as pd
from ftfy import fix_text
import emoji
from tqdm.auto import tqdm
from deep_translator import GoogleTranslator


# ======================= CONFIGURACIÓN =====================
BASE_GLOBAL = Path("/content/drive/MyDrive/todas_las_plataformas")
DESCRIPTION_COL = "description"
NEW_COL = "description_final"
ONLY_THIS_CAREER = None            # p.ej. "Administración_de_Empresas" o None para todas

# Rendimiento
MAX_WORKERS = 2                    # 2–4 si no te limita
CHUNK_LIMIT = 4500                 # margen seguro (<~5000) por llamada

# Reintentos / control de fallos
FAIL_MARKER = "[GT_FAIL] "         # prefijo cuando falla la traducción
RETRY_ONLY_FAILED_FROM_CSV = True  # al relanzar, solo reintenta FAIL o vacías
RETRY_PREV_FAIL = True             # reintenta en esta sesión aunque esté cacheado como FAIL

# Menos ruido en consola (solo progreso y mensajes clave)
# ===========================================================


# URLs, e-mails, HTML
URL_RE = re.compile(r"(https?://\S+|www\.\S+)")
EMAIL_RE = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w{2,}\b")
HTML_TAG_RE = re.compile(r"<[^>]+>")

# Espacios/puntuación
WS_RE = re.compile(r"\s+")
SPACE_BEFORE_PUNCT_RE = re.compile(r"\s+([,.;:!?])")
PUNCT_DUP_RE = re.compile(r"([.!?])\1{1,}")     # "!!" -> "!"
DOT_SPACE_FIX_RE = re.compile(r"\s*\.\s*")       # normalizar espacios alrededor de "."

# Markdown/ruido
MD_STRONG_EM_RE = re.compile(r"(\*\*|__)+")      # **, __
MD_INLINE_EM_RE = re.compile(r"(^|[\s\W])(\*|_)(.+?)(\2)($|[\s\W])", re.DOTALL)  # *texto* / _texto_
PIPE_RE = re.compile(r"\s*\|\s*")                # " | " -> ". "
UNDERSCORES_RUN_RE = re.compile(r"_{2,}")        # "__", "____" -> ". "
LEADING_BULLET_RE = re.compile(r"^\s*[•\-\–\—\·\‣\∙\●\○\▪\▫\►\➤]\s*", re.MULTILINE)

# Encabezados y hashes (#, ##, …)
HASH_HEADING_RE = re.compile(r"(?m)^\s{0,3}#{1,6}\s*")  # quitar encabezados Markdown
HASH_INLINE_RE = re.compile(r"#+")

# --- Variantes de Q&A ---
QA_VARIANTS = [
    re.compile(r'(?i)\bq\s*&\s*a\b'),          # Q & A
    re.compile(r'(?i)\bq\s*/\s*a\b'),          # Q/A
    re.compile(r'(?i)\bq\s*[-–—]\s*a\b'),      # Q-A, Q – A, Q — A
]

def normalize_qa_terms(s: str) -> str:
    # 1) Unificar todas las variantes a "QA"
    for pat in QA_VARIANTS:
        s = pat.sub("QA", s)
    # 2) Expandir "QA" si no está ya expandido (evita duplicar)
    s = re.sub(r'\bQA\b(?!\s*\()', 'QA (Quality Assurance)', s)
    return s

def normalize_unicode(s: str) -> str:
    s = fix_text(s)
    s = html.unescape(s)
    return unicodedata.normalize("NFC", s)

def _strip_markdown_and_layout(s: str) -> str:
    # Quitar HTML, URLs y e-mails
    s = HTML_TAG_RE.sub(" ", s)
    s = URL_RE.sub(" ", s)
    s = EMAIL_RE.sub(" ", s)

    # Quitar emojis
    s = emoji.replace_emoji(s, " ")

    # Quitar viñetas al inicio de línea
    s = LEADING_BULLET_RE.sub("", s)

    # Quitar encabezados y hashes (#)
    s = HASH_HEADING_RE.sub("", s)
    s = HASH_INLINE_RE.sub(" ", s)

    # Quitar negritas/cursivas Markdown ** __ * _
    s = MD_STRONG_EM_RE.sub(" ", s)
    s = MD_INLINE_EM_RE.sub(lambda m: f"{m.group(1)}{m.group(3)}{m.group(5)}", s)

    # Pipes y underscores largos -> separadores ". "
    s = PIPE_RE.sub(". ", s)
    s = UNDERSCORES_RUN_RE.sub(". ", s)

    # Guiones medios/largos como separadores suaves
    s = re.sub(r"\s*[–—]\s*", ". ", s)

    return s

def _fix_spacing_and_punct(s: str) -> str:
    # Quitar espacios antes de puntuación
    s = SPACE_BEFORE_PUNCT_RE.sub(r"\1", s)

    # Normalizar puntos y espacios
    s = DOT_SPACE_FIX_RE.sub(". ", s)     # ". " estándar
    s = re.sub(r"\.\s+\.", ". ", s)       # ".  ." -> ". "

    # Colapsar repeticiones de signos "!!!" -> "!"
    s = PUNCT_DUP_RE.sub(r"\1", s)

    # Si quedó " . " al final -> "."
    s = re.sub(r"\s+\.$", ".", s)

    return s

def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""

    s = normalize_unicode(text)
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    s = s.replace("\n•", "\n")
    s = s.replace("\n", ". ")

    s = _strip_markdown_and_layout(s)

    # >>> normaliza Q&A aquí <<<
    s = normalize_qa_terms(s)

    s = _fix_spacing_and_punct(s)
    s = WS_RE.sub(" ", s).strip()
    return s


# ==================== LECTURA ROBUSTA CSV =====================
def read_csv_robust(path: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, sep=";", low_memory=False)
        except Exception:
            pass
    raise RuntimeError(f"No se pudo leer el CSV: {path}")


# ================== TRADUCTOR POR HILO + CACHÉ =================
_thread_local = threading.local()
_global_cache = {}  # texto_limpio -> traducción o FAIL_MARKER+texto

def _get_translator():
    gt = getattr(_thread_local, "gt", None)
    if gt is None:
        gt = GoogleTranslator(source="auto", target="es")
        _thread_local.gt = gt
    return gt

def _split_into_chunks(text: str, limit: int = CHUNK_LIMIT):
    """Divide en <=limit, intenta cortar por final de frase; si no, corta duro."""
    if len(text) <= limit:
        return [text]
    parts = re.split(r'(?<=[\.\!\?\;:])\s+', text)
    chunks, cur = [], ""
    for p in parts:
        if not p:
            continue
        add = len(p) + (1 if cur else 0)
        if len(cur) + add <= limit:
            cur = f"{cur} {p}".strip() if cur else p
        else:
            if cur:
                chunks.append(cur)
            if len(p) > limit:
                for i in range(0, len(p), limit):
                    chunks.append(p[i:i+limit])
                cur = ""
            else:
                cur = p
    if cur:
        chunks.append(cur)
    return chunks

def _translate_text_with_fail(text: str) -> str:
    """
    Traduce con troceo + 3 reintentos por trozo.
    Si cualquier trozo falla, devuelve FAIL_MARKER + texto original.
    """
    if not text:
        return ""
    # caché de sesión
    cached = _global_cache.get(text)
    if cached is not None:
        if RETRY_PREV_FAIL and isinstance(cached, str) and cached.startswith(FAIL_MARKER):
            pass  # reintenta en esta sesión
        else:
            return cached

    gt = _get_translator()
    outs = []
    for ch in _split_into_chunks(text, CHUNK_LIMIT):
        ok = False
        delay = 2.0
        for _ in range(3):
            try:
                out = gt.translate(ch)
                if not isinstance(out, str) or out.strip() == "":
                    raise RuntimeError("empty")
                outs.append(out)
                ok = True
                break
            except Exception:
                time.sleep(delay)  # backoff 2s, 4s, 8s
                delay *= 2
        if not ok:
            fail_val = f"{FAIL_MARKER}{text}"
            _global_cache[text] = fail_val
            return fail_val

    final = " ".join(outs)
    _global_cache[text] = final
    return final


# ===== TRADUCIR SOLO ÚNICOS (omite vacíos) + MAPEO RÁPIDO =====
def translate_series_unique_multithread(series: pd.Series, max_workers: int = 3) -> pd.Series:
    """
    - Traduce solo valores únicos no vacíos ('' se omiten y se dejan tal cual).
    - Usa hilos y caché en memoria para acelerar.
    """
    texts = series.fillna("").astype(str)

    # únicos no vacíos
    uniques = [u for u in texts.unique().tolist() if u != ""]

    # únicos realmente pendientes (no en caché, o FAIL y queremos reintentar)
    to_do = []
    for u in uniques:
        v = _global_cache.get(u)
        if v is None:
            to_do.append(u)
        elif RETRY_PREV_FAIL and isinstance(v, str) and v.startswith(FAIL_MARKER):
            to_do.append(u)

    if to_do:
        def worker(u):
            return u, _translate_text_with_fail(u)
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            for u, out in tqdm(ex.map(worker, to_do), total=len(to_do),
                               desc="Traduciendo únicos", unit="texto"):
                _global_cache[u] = out

    # mapear: vacíos quedan como "", el resto toma de la caché
    return texts.map(lambda x: _global_cache.get(x, x))


# ===================== PROCESAMIENTO POR CSV ====================
def process_file(target_file: Path):
    try:
        df = read_csv_robust(target_file)
    except Exception as e:
        print(f"[ERROR] Leyendo {target_file.name}: {e}")
        return

    if DESCRIPTION_COL not in df.columns:
        print(f"[WARN] No hay columna '{DESCRIPTION_COL}' en {target_file.name}")
        return

    total = len(df)
    print(f"\n[INFO] Procesando {target_file.name} ({total} filas)")

    cleaned = df[DESCRIPTION_COL].fillna("").apply(clean_text)
    mask_desc_empty = cleaned.eq("")  # filas sin descripción -> no tocar

    if NEW_COL in df.columns and RETRY_ONLY_FAILED_FROM_CSV:
        col = df[NEW_COL].astype(str) if NEW_COL in df.columns else pd.Series("", index=df.index)
        mask_fail  = col.str.startswith(FAIL_MARKER, na=False)
        mask_empty_final = df[NEW_COL].isna() | df[NEW_COL].eq("") if NEW_COL in df.columns else pd.Series(True, index=df.index)
        # pendientes = filas con descripción (no vacías) y final FAIL o vacío
        mask_pending = (~mask_desc_empty) & (mask_fail | mask_empty_final)

        n_pending = int(mask_pending.sum())
        n_skip = int(len(df) - n_pending)
        print(f"[INFO] Reintentando solo pendientes (con descripción): {n_pending} filas | conservando {n_skip} restantes")

        if n_pending > 0:
            translated_pending = translate_series_unique_multithread(
                cleaned[mask_pending], max_workers=MAX_WORKERS
            )
            if NEW_COL not in df.columns:
                df[NEW_COL] = ""
            df.loc[mask_pending, NEW_COL] = translated_pending
        else:
            print("[INFO] No hay pendientes que reintentar.")
    else:
        # Pase completo: traducir solo filas con descripción
        translated = translate_series_unique_multithread(
            cleaned[~mask_desc_empty], max_workers=MAX_WORKERS
        )
        # crear/actualizar solo en las filas con descripción
        if NEW_COL not in df.columns:
            df[NEW_COL] = ""
        df.loc[~mask_desc_empty, NEW_COL] = translated

    # Guardar
    try:
        df.to_csv(target_file, index=False, encoding="utf-8")
    except Exception:
        df.to_csv(target_file, index=False, encoding="utf-8-sig")

    print(f"[OK] Guardado en {target_file}\n")


# ===================== RECORRER TODAS LAS CARRERAS ====================
def process_all():
    if not BASE_GLOBAL.exists():
        raise FileNotFoundError(f"No existe la ruta base: {BASE_GLOBAL}")

    carrera_dirs = [p for p in BASE_GLOBAL.iterdir() if p.is_dir()]
    if ONLY_THIS_CAREER:
        carrera_dirs = [d for d in carrera_dirs if d.name == ONLY_THIS_CAREER]
    if not carrera_dirs:
        print("[WARN] No se encontraron carpetas de carrera para procesar.")
        return

    for carrera_dir in sorted(carrera_dirs, key=lambda x: x.name.lower()):
        expected_name = f"{carrera_dir.name}_Merged.csv"
        target_file = carrera_dir / expected_name
        if not target_file.exists():
            print(f"[SKIP] No existe {expected_name} en {carrera_dir.name}")
            continue
        process_file(target_file)


# =========================== EJECUCIÓN ==========================
process_all()



[INFO] Procesando Administración_de_Empresas_Merged.csv (5220 filas)
[INFO] Reintentando solo pendientes (con descripción): 39 filas | conservando 5181 restantes


Traduciendo únicos:   0%|          | 0/37 [00:00<?, ?texto/s]

[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Administración_de_Empresas/Administración_de_Empresas_Merged.csv


[INFO] Procesando Agroindustria_Merged.csv (1688 filas)
[INFO] Reintentando solo pendientes (con descripción): 19 filas | conservando 1669 restantes


Traduciendo únicos:   0%|          | 0/15 [00:00<?, ?texto/s]

[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Agroindustria/Agroindustria_Merged.csv


[INFO] Procesando Ciencia_de_Datos_Merged.csv (4836 filas)
[INFO] Reintentando solo pendientes (con descripción): 0 filas | conservando 4836 restantes
[INFO] No hay pendientes que reintentar.
[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Ciencia_de_Datos/Ciencia_de_Datos_Merged.csv


[INFO] Procesando Computación_Merged.csv (4101 filas)
[INFO] Reintentando solo pendientes (con descripción): 0 filas | conservando 4101 restantes
[INFO] No hay pendientes que reintentar.
[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Computación/Computación_Merged.csv


[INFO] Procesando Economía_Merged.csv (2506 filas)
[INFO] Reintentando solo pendientes (con descripción): 0 filas | conservando 2506 restantes
[INFO] No hay pendientes que reintentar.
[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Economía/Economía_Merged.csv


[INFO] Procesando Electrici

Traduciendo únicos:   0%|          | 0/1 [00:00<?, ?texto/s]

[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Sistemas_de_Información/Sistemas_de_Información_Merged.csv


[INFO] Procesando Software_Merged.csv (5873 filas)
[INFO] Reintentando solo pendientes (con descripción): 0 filas | conservando 5873 restantes
[INFO] No hay pendientes que reintentar.
[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Software/Software_Merged.csv


[INFO] Procesando Tecnologías_de_la_Información_Merged.csv (4230 filas)
[INFO] Reintentando solo pendientes (con descripción): 0 filas | conservando 4230 restantes
[INFO] No hay pendientes que reintentar.
[OK] Guardado en /content/drive/MyDrive/todas_las_plataformas/Tecnologías_de_la_Información/Tecnologías_de_la_Información_Merged.csv


[INFO] Procesando Telecomunicaciones_Merged.csv (2935 filas)
[INFO] Reintentando solo pendientes (con descripción): 0 filas | conservando 2935 restantes
[INFO] No hay pendientes que reintentar.
[OK] Guardado en /content/drive/MyDrive/todas_las_platafor

In [11]:


import re, html, unicodedata
from pathlib import Path
import pandas as pd
from langdetect import detect, DetectorFactory
from ftfy import fix_text

# -------------------- CONFIG --------------------
BASE_GLOBAL = Path("/content/drive/MyDrive/todas_las_plataformas")
DESCRIPTION_COL = "description"
FINAL_COL = "description_final"
ONLY_THIS_CAREER = None          # p.ej., "Administración_de_Empresas" o None para todas
FAIL_MARKER = "[GT_FAIL] "

# Umbrales
MIN_LEN_FOR_LANGCHECK = 25       # mínimo para confiar en el detector
MIN_SOFTBLOCK_LEN = 120          # si final≈entrada y es largo => soft-block

DetectorFactory.seed = 0         # resultados deterministas

# -------------------- Utils --------------------
def nfc(s: str) -> str:
    if not isinstance(s, str):
        return ""
    return unicodedata.normalize("NFC", html.unescape(fix_text(s))).strip()

WS_RE = re.compile(r"\s+")
PUNCT_RE = re.compile(r"[^\wáéíóúüñÁÉÍÓÚÜÑ]+", flags=re.UNICODE)

def normalize_for_compare(s: str) -> str:
    s = nfc(s).lower()
    s = PUNCT_RE.sub(" ", s)
    s = WS_RE.sub(" ", s)
    return s.strip()

def detect_lang_safe(text: str) -> str:
    try:
        return detect(text) if text and text.strip() else "unknown"
    except Exception:
        return "unknown"

def read_csv_robust(path: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, sep=";", low_memory=False)
        except Exception:
            pass
    raise RuntimeError(f"No se pudo leer el CSV: {path}")

# -------------------- Regla principal --------------------
def needs_ticket(src_raw: str, final_raw: str) -> tuple[bool, str, str]:
    """
    Devuelve (marcar?, motivo, payload_para_ticket).
    payload_para_ticket = texto original limpio que pondremos detrás de [GT_FAIL]
    """
    src = nfc(src_raw)
    final = nfc(final_raw)

    # Sin descripción de origen: no hay nada que traducir
    if src == "":
        return (False, "no_src", "")

    # Ya tenía ticket: no tocar
    if isinstance(final_raw, str) and final_raw.startswith(FAIL_MARKER):
        return (False, "already_fail", "")

    # Final vacío (habiendo origen) → marcar
    if final == "":
        return (True, "empty_final", src)

    # Detectar idiomas (solo si hay suficiente longitud para fiarse)
    lang_src = detect_lang_safe(src)   if len(src)   >= MIN_LEN_FOR_LANGCHECK else "unknown"
    lang_fin = detect_lang_safe(final) if len(final) >= MIN_LEN_FOR_LANGCHECK else "unknown"

    # Soft-block: SOLO marcar si final≈source Y el origen NO es español.
    # (Si el origen ya estaba en español, dejarlo pasar aunque sean casi iguales.)
    if len(final) >= MIN_SOFTBLOCK_LEN:
        if normalize_for_compare(final) == normalize_for_compare(src):
            if lang_src != "es" and lang_fin != "es":
                return (True, "soft_block_like_src", src)

    # Idioma final claramente NO español → marcar (multi-idioma)
    if len(final) >= MIN_LEN_FOR_LANGCHECK and lang_fin not in ("es", "unknown"):
        return (True, f"lang_{lang_fin}", src)

    # Por defecto OK (incluye español, unknown o textos cortos)
    return (False, "ok", "")

# -------------------- Auditoría por archivo --------------------
def audit_file(path: Path):
    try:
        df = read_csv_robust(path)
    except Exception as e:
        print(f"[ERROR] Leyendo {path.name}: {e}")
        return

    if DESCRIPTION_COL not in df.columns:
        print(f"[WARN] No hay columna '{DESCRIPTION_COL}' en {path.name}")
        return
    if FINAL_COL not in df.columns:
        print(f"[WARN] No hay columna '{FINAL_COL}' en {path.name} (nada que auditar)")
        return

    src = df[DESCRIPTION_COL].fillna("").astype(str)
    fin = df[FINAL_COL].fillna("").astype(str)

    to_mark_idx = []
    payloads = {}

    # 1ª pasada: decide rápido qué marcar
    for i, (s, f) in enumerate(zip(src, fin)):
        mark, _, payload = needs_ticket(s, f)
        if mark:
            to_mark_idx.append(i)
            payloads[i] = payload

    n_mark = len(to_mark_idx)
    if n_mark > 0:
        # Aplica el ticket: [GT_FAIL] + texto original limpio
        for i in to_mark_idx:
            df.at[i, FINAL_COL] = FAIL_MARKER + payloads[i]

    total = len(df)
    print(f"[INFO] {path.name} → filas:{total} | marcadas:{n_mark}")

    # Guardar en el MISMO CSV
    try:
        df.to_csv(path, index=False, encoding="utf-8")
    except Exception:
        df.to_csv(path, index=False, encoding="utf-8-sig")

# -------------------- Recorrer todas las carreras --------------------
def audit_all():
    base = BASE_GLOBAL
    if not base.exists():
        raise FileNotFoundError(f"No existe la ruta base: {base}")

    dirs = [p for p in base.iterdir() if p.is_dir()]
    if ONLY_THIS_CAREER:
        dirs = [d for d in dirs if d.name == ONLY_THIS_CAREER]
    if not dirs:
        print("[WARN] No se encontraron carpetas.")
        return

    for d in sorted(dirs, key=lambda x: x.name.lower()):
        expected = d / f"{d.name}_Merged.csv"
        if not expected.exists():
            print(f"[SKIP] No existe {expected.name} en {d.name}")
            continue
        audit_file(expected)

# -------------------------- EJECUCIÓN --------------------------
audit_all()


[INFO] Administración_de_Empresas_Merged.csv → filas:5220 | marcadas:38
[INFO] Agroindustria_Merged.csv → filas:1688 | marcadas:19


KeyboardInterrupt: 

In [14]:
# ======= BORRAR FILAS SIN CONTENIDO EN description NI skills (considera "[]") =======
from pathlib import Path
import re
import pandas as pd

# -------- Config --------
BASE_GLOBAL = Path("/content/drive/MyDrive/todas_las_plataformas")
DESCRIPTION_COL = "description"
SKILLS_COL = "skills"
ONLY_THIS_CAREER = None  # p.ej. "Administración_de_Empresas" o None para todas

# -------- Lectura robusta --------
def read_csv_robust(path: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, sep=";", low_memory=False)
        except Exception:
            pass
    raise RuntimeError(f"No se pudo leer el CSV: {path}")

# -------- Criterio de "vacío" --------
BRACKETS_EMPTY_RE = re.compile(r"^\s*\[\s*\]\s*$")  # coincide con [], [   ], etc.

def series_is_empty_like(s: pd.Series) -> pd.Series:
    """
    Vacío si:
      - NaN
      - cadena vacía / solo espacios
      - exactamente "[]", "[   ]", etc.
    """
    if s is None:
        # si la columna no existe, considérala toda vacía
        return pd.Series(True, index=pd.RangeIndex(0))
    # Convertimos a string preservando NaN para chequearlos explícitamente
    is_na = s.isna()
    as_str = s.astype(str)
    is_blank = as_str.str.strip().eq("")
    is_brackets = as_str.str.match(BRACKETS_EMPTY_RE)
    return is_na | is_blank | is_brackets

# -------- Proceso por archivo --------
def clean_file(file_path: Path):
    try:
        df = read_csv_robust(file_path)
    except Exception as e:
        print(f"[ERROR] {file_path.name}: {e}")
        return

    n_before = len(df)
    if n_before == 0:
        print(f"[SKIP] {file_path.name}: vacío.")
        return

    # Si faltan columnas, las tratamos como vacías
    desc = df[DESCRIPTION_COL] if DESCRIPTION_COL in df.columns else pd.Series([None]*n_before)
    skills = df[SKILLS_COL] if SKILLS_COL in df.columns else pd.Series([None]*n_before)

    desc_empty = series_is_empty_like(desc)
    skills_empty = series_is_empty_like(skills)

    # Mantener filas donde AL MENOS una columna tiene contenido
    keep_mask = ~(desc_empty & skills_empty)
    df_kept = df[keep_mask].copy()

    removed = n_before - len(df_kept)
    if removed > 0:
        try:
            df_kept.to_csv(file_path, index=False, encoding="utf-8")
        except Exception:
            df_kept.to_csv(file_path, index=False, encoding="utf-8-sig")
        print(f"[OK] {file_path.name}: eliminadas {removed} filas (quedan {len(df_kept)}).")
    else:
        print(f"[OK] {file_path.name}: nada que eliminar ({n_before} filas).")

# -------- Recorrer todas las carreras --------
def run_all():
    base = BASE_GLOBAL
    if not base.exists():
        raise FileNotFoundError(f"No existe la ruta base: {base}")

    carreras = [p for p in base.iterdir() if p.is_dir()]
    if ONLY_THIS_CAREER:
        carreras = [d for d in carreras if d.name == ONLY_THIS_CAREER]
    if not carreras:
        print("[WARN] No se encontraron carpetas de carrera.")
        return

    for d in sorted(carreras, key=lambda x: x.name.lower()):
        target = d / f"{d.name}_Merged.csv"
        if not target.exists():
            print(f"[SKIP] No existe {target.name} en {d.name}")
            continue
        clean_file(target)

# -------- Ejecutar --------
run_all()


[OK] Administración_de_Empresas_Merged.csv: eliminadas 384 filas (quedan 4836).
[OK] Agroindustria_Merged.csv: eliminadas 1 filas (quedan 1687).
[OK] Ciencia_de_Datos_Merged.csv: eliminadas 206 filas (quedan 4630).
[OK] Computación_Merged.csv: nada que eliminar (4101 filas).
[OK] Economía_Merged.csv: nada que eliminar (2506 filas).
[OK] Electricidad_Merged.csv: nada que eliminar (2066 filas).
[OK] Electrónica_y_Automatización_Merged.csv: nada que eliminar (6349 filas).
[OK] Física_Merged.csv: nada que eliminar (1614 filas).
[OK] Geología_Merged.csv: nada que eliminar (1074 filas).
[OK] Ingeniería_Ambiental_Merged.csv: nada que eliminar (3295 filas).
[OK] Ingeniería_Civil_Merged.csv: nada que eliminar (3864 filas).
[OK] Ingeniería_de_la_Producción_Merged.csv: nada que eliminar (4503 filas).
[OK] Ingeniería_Química_Merged.csv: nada que eliminar (1148 filas).
[OK] Inteligencia_Artificial_Merged.csv: nada que eliminar (6805 filas).
[OK] Matemática_Merged.csv: nada que elimina

In [15]:
# ========== NORMALIZAR 'description_final' EN *_Merged.csv (versión robusta) ==========
# Limpia HTML/URLs/emails/emojis, viñetas en TODO el texto, markdown suelto,
# separadores decorativos, tags de idioma ([SPANISH], EN:, etc.), pipes/underscores,
# colapsa ". ." repetidos, corrige espacios y puntuación.
# NO modifica filas vacías ni con [GT_FAIL].

import re, html, unicodedata
from pathlib import Path
import pandas as pd
from ftfy import fix_text
import emoji

# ------------------- CONFIG -------------------
BASE_GLOBAL = Path("/content/drive/MyDrive/todas_las_plataformas")
FINAL_COL = "description_final"
ONLY_THIS_CAREER = None          # p.ej. "Administración_de_Empresas" o None
FAIL_MARKER = "[GT_FAIL]"

# ----------------- REGEX / UTILS --------------
URL_RE = re.compile(r"(https?://\S+|www\.\S+)")
EMAIL_RE = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w{2,}\b")
HTML_TAG_RE = re.compile(r"<[^>]+>")

WS_RE = re.compile(r"\s+")
SPACE_BEFORE_PUNCT_RE = re.compile(r"\s+([,.;:!?])")
PUNCT_DUP_RE = re.compile(r"([.!?])\1{1,}")        # "!!!" -> "!"
DOT_SPACE_FIX_RE = re.compile(r"\s*\.\s*")         # normaliza espacios alrededor de "."
MULTI_DOTS_RE = re.compile(r"(?:\.\s*){2,}")       # ". ." repetidos -> ". "
LEADING_PUNCT_RE = re.compile(r"^[\.\,\;\:\-\–\—\·\•\*]+")  # puntos/viñetas al inicio

# Markdown y decoración
MD_STRONG_EM_RE = re.compile(r"(\*\*|__)+")        # **, __
MD_INLINE_EM_RE = re.compile(r"(^|[\s\W])(\*|_)(.+?)(\2)($|[\s\W])", re.DOTALL)  # *texto* / _texto_
STRAY_STARS_RE = re.compile(r"\*+")                # asteriscos sueltos -> espacio
PIPE_RE = re.compile(r"\s*\|\s*")                  # " | " -> ". "
UNDERSCORES_RUN_RE = re.compile(r"_{2,}")          # "__", "____" -> ". "
DECORATIVE_LINE_RE = re.compile(r"(?m)^\s*([=\-_*~<>\.]{3,})\s*$")  # líneas de ====, ----, ****, ...

# Viñetas
BULLET_CHARS = "•◦·‣∙●○▪▫◆◇■□►➤–—"  # incluye guiones largos/medios
LEADING_BULLET_RE = re.compile(r"^\s*[•\-\–\—\·\‣\∙\●\○\▪\▫\►\➤]\s*", re.MULTILINE)

# Tags de idioma / encabezados tipo idioma
LANG_TAG_INLINE_RE = re.compile(r"\[\s*(spanish|english|es|en|pt|fr|de|it)\s*\]", re.I)
LANG_TAG_LINE_RE = re.compile(r"(?m)^\s*(spanish|english|es|en|pt|fr|de|it)\s*:\s*", re.I)

# Q&A → QA (Quality Assurance)
QA_VARIANTS = [
    re.compile(r'(?i)\bq\s*&\s*a\b'),
    re.compile(r'(?i)\bq\s*/\s*a\b'),
    re.compile(r'(?i)\bq\s*[-–—]\s*a\b'),
    re.compile(r'(?i)\bq\s*\+\s*a\b'),
    re.compile(r'(?i)\bq\s*y\s*a\b'),
]

def nfc(s: str) -> str:
    s = fix_text(s or "")
    s = html.unescape(s)
    return unicodedata.normalize("NFC", s)

def _strip_markdown_and_layout(s: str) -> str:
    # HTML / URLs / Emails / Emojis
    s = HTML_TAG_RE.sub(" ", s)
    s = URL_RE.sub(" ", s)
    s = EMAIL_RE.sub(" ", s)
    s = emoji.replace_emoji(s, " ")

    # Elimina tags de idioma y líneas decorativas
    s = LANG_TAG_INLINE_RE.sub(" ", s)
    s = LANG_TAG_LINE_RE.sub("", s)
    s = DECORATIVE_LINE_RE.sub(" ", s)

    # Viñetas: al inicio de línea y en todo el texto
    s = LEADING_BULLET_RE.sub("", s)
    s = s.translate({ord(ch): " " for ch in BULLET_CHARS})

    # Encabezados y hashes (#)
    s = re.sub(r"(?m)^\s{0,3}#{1,6}\s*", "", s)  # #, ## al inicio
    s = re.sub(r"#+", " ", s)                    # hashes sueltos

    # Markdown ** __ * _
    s = MD_STRONG_EM_RE.sub(" ", s)
    s = MD_INLINE_EM_RE.sub(lambda m: f"{m.group(1)}{m.group(3)}{m.group(5)}", s)
    s = STRAY_STARS_RE.sub(" ", s)               # quita * sueltos

    # Pipes y underscores largos -> separadores
    s = PIPE_RE.sub(". ", s)
    s = UNDERSCORES_RUN_RE.sub(". ", s)

    # Guiones medios/largos como separadores suaves
    s = re.sub(r"\s*[–—]\s*", ". ", s)

    return s

def _normalize_qa_terms(s: str) -> str:
    for pat in QA_VARIANTS:
        s = pat.sub("QA", s)
    s = re.sub(r'\bQA\b(?!\s*\()', 'QA (Quality Assurance)', s)
    return s

def _fix_spacing_and_punct(s: str) -> str:
    s = SPACE_BEFORE_PUNCT_RE.sub(r"\1", s)
    s = DOT_SPACE_FIX_RE.sub(". ", s)      # ". " estándar
    s = MULTI_DOTS_RE.sub(". ", s)         # colapsa ". .", ". . .", etc.
    s = re.sub(r"\.\s+\.", ". ", s)        # ".  ." -> ". "
    s = PUNCT_DUP_RE.sub(r"\1", s)         # "!!!" -> "!"
    s = LEADING_PUNCT_RE.sub("", s)        # quita puntuación/viñetas líderes
    s = re.sub(r"\s+\.$", ".", s)          # " . " final -> "."
    return s

def clean_final_text(text: str) -> str:
    """Normaliza SOLO el contenido de description_final ya traducido."""
    if not isinstance(text, str):
        return ""
    s = nfc(text)

    # Saltos de línea -> ". "
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    s = s.replace("\n•", "\n")
    s = s.replace("\n", ". ")

    # Ruido y maquetación
    s = _strip_markdown_and_layout(s)

    # Q&A → QA (Quality Assurance)
    s = _normalize_qa_terms(s)

    # Espaciado y puntuación
    s = _fix_spacing_and_punct(s)

    # Compactar espacios finales
    s = WS_RE.sub(" ", s).strip()

    # Si quedó algo como ". Texto" por limpieza, vuelve a limpiar líder
    s = LEADING_PUNCT_RE.sub("", s).strip()
    return s

# --------------- LECTURA ROBUSTA ----------------
def read_csv_robust(path: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, sep=";", low_memory=False)
        except Exception:
            pass
    raise RuntimeError(f"No se pudo leer el CSV: {path}")

# --------------- PROCESO POR ARCHIVO ---------------
def normalize_file(path: Path):
    try:
        df = read_csv_robust(path)
    except Exception as e:
        print(f"[ERROR] Leyendo {path.name}: {e}")
        return

    if FINAL_COL not in df.columns:
        print(f"[WARN] No hay columna '{FINAL_COL}' en {path.name}")
        return

    col = df[FINAL_COL].astype(str)

    # NO tocar vacíos ni filas con ticket
    mask_keep = col.str.strip().eq("") | col.str.startswith(FAIL_MARKER, na=False)
    mask_norm = ~mask_keep

    n_total = len(df)
    n_norm = int(mask_norm.sum())
    if n_norm == 0:
        print(f"[OK] {path.name}: nada que normalizar ({n_total} filas).")
        return

    df.loc[mask_norm, FINAL_COL] = col[mask_norm].map(clean_final_text)

    # Guardar
    try:
        df.to_csv(path, index=False, encoding="utf-8")
    except Exception:
        df.to_csv(path, index=False, encoding="utf-8-sig")

    print(f"[OK] {path.name}: normalizadas {n_norm}/{n_total} filas.")

# --------------- RECORRER TODAS LAS CARRERAS ---------------
def normalize_all():
    base = BASE_GLOBAL
    if not base.exists():
        raise FileNotFoundError(f"No existe la ruta base: {base}")

    dirs = [p for p in base.iterdir() if p.is_dir()]
    if ONLY_THIS_CAREER:
        dirs = [d for d in dirs if d.name == ONLY_THIS_CAREER]
    if not dirs:
        print("[WARN] No se encontraron carpetas.")
        return

    for d in sorted(dirs, key=lambda x: x.name.lower()):
        target = d / f"{d.name}_Merged.csv"
        if not target.exists():
            print(f"[SKIP] No existe {target.name} en {d.name}")
            continue
        normalize_file(target)

# --------------------- EJECUCIÓN ---------------------
normalize_all()


[OK] Administración_de_Empresas_Merged.csv: normalizadas 4835/4836 filas.
[OK] Agroindustria_Merged.csv: normalizadas 1687/1687 filas.
[OK] Ciencia_de_Datos_Merged.csv: normalizadas 4630/4630 filas.
[OK] Computación_Merged.csv: normalizadas 4099/4101 filas.
[OK] Economía_Merged.csv: normalizadas 2506/2506 filas.
[OK] Electricidad_Merged.csv: normalizadas 2066/2066 filas.
[OK] Electrónica_y_Automatización_Merged.csv: normalizadas 6349/6349 filas.
[OK] Física_Merged.csv: normalizadas 1614/1614 filas.
[OK] Geología_Merged.csv: normalizadas 1074/1074 filas.
[OK] Ingeniería_Ambiental_Merged.csv: normalizadas 3295/3295 filas.
[OK] Ingeniería_Civil_Merged.csv: normalizadas 3864/3864 filas.
[OK] Ingeniería_de_la_Producción_Merged.csv: normalizadas 4503/4503 filas.
[OK] Ingeniería_Química_Merged.csv: normalizadas 1148/1148 filas.
[OK] Inteligencia_Artificial_Merged.csv: normalizadas 6805/6805 filas.
[OK] Matemática_Merged.csv: normalizadas 399/399 filas.
[OK] Matemática_Aplicada_